In [ ]:
!pip install opencv-python

In [9]:
import cv2
import numpy as np
import time
import psutil

# Load desired gesture representation (image or video clip)
desired_gesture = cv2.imread('/content/Screenshot 2024-04-09 212336.png')  # Update the path with your desired gesture image

# Function to detect gesture in a frame
def detect_gesture(frame, gesture_template):
    # Convert frame and template to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_template = cv2.cvtColor(gesture_template, cv2.COLOR_BGR2GRAY)

    # Perform template matching
    result = cv2.matchTemplate(gray_frame, gray_template, cv2.TM_CCOEFF_NORMED)

    # Set threshold
    threshold = 0.7  # Adjust threshold as needed

    # Find locations where the correlation coefficient is above the threshold
    loc = np.where(result >= threshold)

    return loc, np.max(result)  # Return locations and maximum correlation coefficient

# Function to annotate frame with "DETECTED" text, accuracy, detection time, and resource usage
def annotate_frame(frame, accuracy, detection_time, cpu_usage, memory_usage):
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_color = (0, 255, 0)  # Green color
    corner_position = (20, 50)  # Position of the text in the top-left corner

    # Annotate the frame with "DETECTED" text, accuracy, detection time, and resource usage
    text = f'DETECTED (Accuracy: {accuracy:.2f}, Time: {detection_time:.2f} secs, CPU: {cpu_usage:.2f}%, Memory: {memory_usage:.2f}%)'
    cv2.putText(frame, text, corner_position, font, font_scale, text_color, font_thickness)

# Function to draw bounding box around detected gesture
def draw_bounding_box(frame, loc):
    for pt in zip(*loc[::-1]):
        cv2.rectangle(frame, pt, (pt[0] + desired_gesture.shape[1], pt[1] + desired_gesture.shape[0]), (0, 255, 0), 2)

# Read test video
test_video_path = '/content/WIN_20240409_20_55_43_Pro.mp4'  # Update the path with your test video
cap = cv2.VideoCapture(test_video_path)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video = cv2.VideoWriter('/content/output_video3.avi', fourcc, fps, (frame_width, frame_height))

# Initialize variables for timing
start_time = time.time()
previous_time = start_time

# Loop through video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Detect gesture in the frame
    gesture_loc, accuracy = detect_gesture(frame, desired_gesture)

    # Calculate detection time
    current_time = time.time()
    detection_time = current_time - previous_time
    previous_time = current_time

    # Get CPU usage
    cpu_usage = psutil.cpu_percent()

    # Get memory usage
    memory_usage = psutil.virtual_memory().percent

    # If gesture is detected, annotate the frame
    if gesture_loc[0].size > 0:
        annotate_frame(frame, accuracy, detection_time, cpu_usage, memory_usage)
        draw_bounding_box(frame, gesture_loc)

    # Write annotated frame to output video
    output_video.write(frame)

# Release resources
cap.release()
output_video.release()
cv2.destroyAllWindows()
